In [5]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [7]:
from apad_env import APADEnv
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback, CallbackList
import time
import numpy as np

In [8]:
class TimerCallback(BaseCallback):
    def __init__(self):
        super().__init__()
        self.start_time = time.time()
    
    def _on_step(self):
        if self.num_timesteps % 1000 == 0:
            elapsed = time.time() - self.start_time
            rate = self.num_timesteps / elapsed
            remaining = (self.locals['total_timesteps'] - self.num_timesteps) / rate
            print(f"Step {self.num_timesteps}, {elapsed:.0f}s elapsed, {remaining:.0f}s remaining")
        return True

# Train

In [13]:
env = APADEnv()

model = DQN(
    "MlpPolicy", 
    env,  
    exploration_initial_eps=1.0,    # Start with 100% random
    exploration_final_eps=0.1,      # End with 10% random  
    exploration_fraction=0.5,       # Take half of training to decay
    learning_rate=1e-3,             # Slightly higher learning rate
    tensorboard_log="./dqn_logs/",
    verbose=2
)

model.learn(total_timesteps=100000, callback=TimerCallback())

#eval_callback = EvalCallback(
#    env, 
#    eval_freq=1000,
#    #log_path="./logs/",
#    #best_model_save_path="./models/"
#)

#model.learn(total_timesteps=80000, callback=eval_callback)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./dqn_logs/DQN_42
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 14.2     |
|    ep_rew_mean      | -3.05    |
|    exploration_rate | 0.999    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 11665    |
|    time_elapsed     | 0        |
|    total_timesteps  | 57       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 27.5     |
|    ep_rew_mean      | -7.84    |
|    exploration_rate | 0.996    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2892     |
|    time_elapsed     | 0        |
|    total_timesteps  | 220      |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.231    |
|    n_updates        | 29       |
--

In [14]:
model.save("apad_dqn_model")